## Uncertainty Quantification for fine-tuned models

Let's repeat the exercise for the case where you are fine-tuning an existing model. 

Before anything, you must download and export the PET-MAD checkpoint that will be used in this tutorial (you can also copy from another directory, if you've downloaded and exported the model already).

```
wget https://huggingface.co/lab-cosmo/pet-mad/resolve/v1.0.2/models/pet-mad-v1.0.2.ckpt
mtt export pet-mad-v1.0.2.ckpt -o pet-mad.pt
```

In [ ]:
import ase.build
import ase.io
import matplotlib.pyplot as plt
import numpy as np

from ase.calculators.emt import EMT

calculator = EMT()
structure = ase.build.bulk("Al", "fcc", cubic=True)

Same as before, we generate 1000 structures and compute the energies using the effective medium potential (EMT), then save the datasets:

In [ ]:
structures = []
for i in range(1000):
    s = structure.copy()
    s.rattle(0.3, seed=i)
    s.calc = calculator
    s.info["energy"] = s.get_potential_energy()
    s.arrays["forces"] = s.get_forces()
    s.info["stress"] = s.get_stress(voigt=False)
    s.calc = None
    structures.append(s)

ase.io.write("dataset.xyz", structures[:100])
ase.io.write("evaluation.xyz", structures[100:])

You can now launch the model fine-tuning, LLPR training, and evaluation processes with the `metatrain` CLI.

_Before running the below commands, **take a moment to study the YAML files being used** and make sure you understand the options being passed to `metatrain`_:

```
mtt train options.yaml
mtt train options-llpr.yaml -o model-llpr.pt
mtt eval model-llpr.pt eval.yaml -b 20
```



We plot the spread of actual errors vs. the predicted errors from the LLPR formalism, now for the model fine-tuned from PET-MAD:

In [ ]:
structures = ase.io.read("evaluation.xyz", ":")
evaluated_structures = ase.io.read("output.xyz", ":")

uncertainties = [s.info["energy_uncertainty"] for s in evaluated_structures]
predicted_energies = [s.get_potential_energy() for s in evaluated_structures]
true_energies = [s.get_potential_energy() for s in structures]

# Hard-code the zoomed in region of the plot and iso-lines.
quantile_lines = [0.00916, 0.10256, 0.4309805, 1.71796, 2.5348, 3.44388]
min_val, max_val = 3e-3, 3e-2

import matplotlib.pyplot as plt
plt.figure(figsize=(6, 6))
plt.plot([min_val, max_val], [min_val, max_val], "k--", lw=0.75)
for factor in quantile_lines:
    plt.plot([min_val, max_val], [factor * min_val, factor * max_val], "k:", lw=0.75)
plt.scatter(uncertainties, np.abs(np.array(predicted_energies) - np.array(true_energies)))
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Predicted energy uncertainty (eV)")
plt.ylabel("Absolute error in predicted energy (eV)")
plt.title("Predicted uncertainty vs actual error")
plt.grid()
plt.savefig("uncertainty_vs_error.png", dpi=300)